In [1]:
import pandas as pd
import json

import functools

In [2]:
fns = ['/home/eamonn/Projects/corpuscule/ycac/ycac_all.corpus']

In [3]:
df = pd.DataFrame()

In [4]:
for fn in fns:
    df = df.append(pd.read_csv(fn,encoding='latin1'))

In [5]:
len(df)

12275807

In [6]:
@functools.lru_cache(maxsize=None)
def str_to_vec(s):
    l = eval(s)
    rep = ""
    for i in range(12):
        if i in l:
            rep += "1"
        else:
            rep += "0"
    return rep

In [7]:
df.head(5)

,offset,PCsInNormalForm,file,Composer
0,3.000,[7],Danse Andalouse Eb major.mid,Ascher
1,3.500,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher
2,3.625,[0],Danse Andalouse Eb major.mid,Ascher
3,3.750,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher
4,3.875,[0],Danse Andalouse Eb major.mid,Ascher


In [8]:
%%time
df['canonical'] = df.PCsInNormalForm.apply(str_to_vec)

CPU times: user 1.98 s, sys: 26.7 ms, total: 2 s
Wall time: 2.01 s


In [9]:
df.head(5)

,offset,PCsInNormalForm,file,Composer,canonical
0,3.000,[7],Danse Andalouse Eb major.mid,Ascher,000000010000
1,3.500,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher,100100001000
2,3.625,[0],Danse Andalouse Eb major.mid,Ascher,100000000000
3,3.750,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher,100100001000
4,3.875,[0],Danse Andalouse Eb major.mid,Ascher,100000000000


In [10]:
u = df.canonical.unique()
len(u)

3812

In [11]:
df.head(10).file[1]

'Danse Andalouse Eb major.mid'

In [12]:
meta = pd.read_csv(open('/home/eamonn/Projects/corpuscule/ycac/csv/metadata/YCAC-metadata.csv', 'r'))

In [13]:
meta.Date.fillna(meta.Range, inplace=True)

In [14]:
import numpy as np

def process_range(d):
    try:
        if '-' in d:
            a = int(d.split('-')[0])
            b = int(d.split('-')[1])
            return int((a + b) / 2)
        else:
            return int(d)
    except:
        return np.nan

In [15]:
meta['EstDate'] = meta.Date.apply(process_range)

In [16]:
df['file'] = df.file.apply(lambda s: s.replace('.mid', ''))
df['Filename'] = df['file']

In [17]:
master = pd.merge(df, meta, on='Filename')

In [18]:
master.head(5)

,offset,PCsInNormalForm,file,Composer_x,canonical,Filename,Title,Composer_y,CatNo,Date,...,Genre,Subgenre,Species,Species2,Nationality,Key,Modulation,Folder,Comments,EstDate
0,3.000,[7],Danse Andalouse Eb major,Ascher,000000010000,Danse Andalouse Eb major,Danse Andalouse,Ascher,Op30,1850-1860,...,OtherSolo,NaN,NaN,NaN,England,Eb major,NaN,A,NaN,1855.0
1,3.500,"[8, 0, 3]",Danse Andalouse Eb major,Ascher,100100001000,Danse Andalouse Eb major,Danse Andalouse,Ascher,Op30,1850-1860,...,OtherSolo,NaN,NaN,NaN,England,Eb major,NaN,A,NaN,1855.0
2,3.625,[0],Danse Andalouse Eb major,Ascher,100000000000,Danse Andalouse Eb major,Danse Andalouse,Ascher,Op30,1850-1860,...,OtherSolo,NaN,NaN,NaN,England,Eb major,NaN,A,NaN,1855.0
3,3.750,"[8, 0, 3]",Danse Andalouse Eb major,Ascher,100100001000,Danse Andalouse Eb major,Danse Andalouse,Ascher,Op30,1850-1860,...,OtherSolo,NaN,NaN,NaN,England,Eb major,NaN,A,NaN,1855.0
4,3.875,[0],Danse Andalouse Eb major,Ascher,100000000000,Danse Andalouse Eb major,Danse Andalouse,Ascher,Op30,1850-1860,...,OtherSolo,NaN,NaN,NaN,England,Eb major,NaN,A,NaN,1855.0


## Dump to text file

In [21]:
top_composers = list(master.Composer_x.value_counts().keys()[:50])

In [25]:
for composer in top_composers:
    sub = master[master.Composer_x == composer]
    sub = sub[['file', 'canonical']]

    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(list(zip(files,vecs)),columns = ['file', 'canonical'])
    fv.reset_index()
    
    # done = fv.groupby('file')['canonical'].apply(list)
    docs = ['{} '.format(l) for l in list(fv.canonical)]
    
    del fv

    print("{} group done".format(composer))

    with open('{}.corpus'.format(composer), 'w') as f:
        f.writelines(docs)

    print("{} dump done".format(composer))

Mozart group done
Mozart dump done
Beethoven group done
Beethoven dump done
Haydn group done
Haydn dump done
Schubert group done
Schubert dump done
Scarlatti group done
Scarlatti dump done
Chopin group done
Chopin dump done
Bach group done
Bach dump done
Brahms group done
Brahms dump done
Liszt group done
Liszt dump done
Mendelssohn group done
Mendelssohn dump done
Dvorak group done
Dvorak dump done
Telemann group done
Telemann dump done
Tchaikovsky group done
Tchaikovsky dump done
Handel group done
Handel dump done
Schumann group done
Schumann dump done
Wagner group done
Wagner dump done
Vivaldi group done
Vivaldi dump done
Saint-Saens group done
Saint-Saens dump done
Hummel group done
Hummel dump done
Verdi group done
Verdi dump done
Rossini group done
Rossini dump done
Byrd group done
Byrd dump done
Sor group done
Sor dump done
Weber group done
Weber dump done
Alkan group done
Alkan dump done
Albeniz group done
Albeniz dump done
Czerny group done
Czerny dump done
Faure group done
Fa

In [28]:
start_years = [x for x in range(1650, 1950, 10)]

In [30]:
eras = [(x, x+9) for x in start_years]

In [36]:
eras = [(1650,1850)]

In [37]:
for era in eras:
    start_year = era[0]
    end_year = era[1]
    
    sub = master[master.EstDate.isin(range(start_year, end_year))]
    sub = sub[['file', 'canonical']]

    label = "{}-{}".format(start_year, end_year)
    
    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(list(zip(files,vecs)),columns = ['file', 'canonical'])
    fv.reset_index()
    
    #done = fv.groupby('file')['canonical'].apply(list)
    docs = ['{} '.format(l) for l in list(fv.canonical)]

    print("{} group done".format(label))

    with open('{}.corpus'.format(label), 'w') as f:
        f.writelines(docs)
    
    print("{} dump done".format(label))

1650-1850 group done
1650-1850 dump done
